In [2]:
import re
from langchain_core.documents import Document
from langchain_community.document_loaders import TextLoader
from splitter import *
from retriever import *

# Định nghĩa regex pattern cho các tiêu đề (sections)
patterns = [
    r'(?<=\n)Chương \d+: .+',  # Mục lớn như "Chương 1: TỔNG QUAN VỀ BAO BÌ"
    r'(?<=\n)\d+\.\d+ .+',  # Mục nhỏ hơn như "1.1 ĐẶC ĐIỂM CHUNG"
    r'(?<=\n)\d+\.\d+\.\d+ .+',  # Mục nhỏ hơn nữa như "1.1.1 Đặc điểm chung của bao bì"
    r'(?<=\n)\d+\.\d+\.\d+\.\d+ .+'  # Mục nhỏ hơn nữa như "1.1.2.1 Tính năng chứa đựng sản phẩm"
]

# Chia tài liệu thành các đoạn
loaders = TextLoader("doc/"+"spkt.txt", encoding = 'UTF-8')
docs = [loaders.load()]
splitter = PatternSplitter(patterns)
split_docs = splitter.split_documents(docs[0])
# In các đoạn
def beautiful_print(split_docs):
    for split_doc in split_docs:
        print(" ".join([split_doc.metadata['chapter'],split_doc.metadata['section'],split_doc.metadata['subsection'],split_doc.metadata['subsubsection'],]))
        if type(split_doc.metadata['summary']) is str:
            print(split_doc.metadata['summary'][:50])
        else:
            print(split_doc.metadata['summary'])
        print("----------")
        beautiful_print(split_doc.metadata["subtree"])

         
beautiful_print(split_docs)


before sum
after sum
end
   
-1
----------
Chương 1: TỔNG QUAN VỀ BAO BÌ   
**Chapter 1: Overview of Packaging**

This chapter
----------
Chương 1: TỔNG QUAN VỀ BAO BÌ 1.1 ĐẶC ĐIỂM CHUNG  
### 1.1 GENERAL CHARACTERISTICS

#### 1.1.1 Genera
----------
Chương 1: TỔNG QUAN VỀ BAO BÌ 1.1 ĐẶC ĐIỂM CHUNG 1.1.1 Đặc điểm chung của bao bì 
**SUMMARY OF PACKAGING OVERVIEW**

**1.1.1 General
----------
Chương 1: TỔNG QUAN VỀ BAO BÌ 1.1 ĐẶC ĐIỂM CHUNG 1.1.2 Tính năng của bao bì 
### 1.1.2 Features of Packaging

Packaging is defi
----------
Chương 1: TỔNG QUAN VỀ BAO BÌ 1.1 ĐẶC ĐIỂM CHUNG 1.1.2 Tính năng của bao bì 1.1.2.1 Tính năng chứa đựng sản phẩm
### 1.1.2.1 Characteristics of the Product

**Gene
----------
Chương 1: TỔNG QUAN VỀ BAO BÌ 1.1 ĐẶC ĐIỂM CHUNG 1.1.2 Tính năng của bao bì 1.1.2.2 Tính năng bảo vệ
### 1.1.2.2 Protective Features of Packaging

Pack
----------
Chương 1: TỔNG QUAN VỀ BAO BÌ 1.1 ĐẶC ĐIỂM CHUNG 1.1.2 Tính năng của bao bì 1.1.2.3 Tính năng thông tin về sản phẩm.
### 1.1.2.3

In [19]:
from util import *
from retriever import *

question = "phạm vi ứng dụng của nhãn hàng trên bao bì"
en_question = vn_2_en(question)
print(en_question)
retriever = TreeRetriever(documents=split_docs)
result = retriever.invoke(en_question)
print(result.page_content)

    scope of application of the brand on packaging
Nhãn hàng: Nhãn hàng cũng là một thành tố của bao bì. Nhãn hàng thường được dán lên các chai lọ nhựa hoặc chai thủy tinh bằng keo dán trên dây chuyền dán nhãn, hoặc sử dụng nhãn tự dính là loại nhãn được in trên decal – một vật liệu có phủ sẵn keo ở mặt sau.
Nhãn có thể in trên vật liệu giấy, nhựa, hoặc vật liệu decal
Sau khi in, thường là in offset tờ rời, nhãn giấy có thể được cắt thành các xấp nhãn nếu nhãn có hình chữ nhật, hoặc được cắt và bế thụt theo từng xấp, nếu nhãn có hình dạng đặc biệt (hình 2.59). Nhăn giấy được giao hàng theo từng xấp và được dán bằng keo dán trên các dây chuyền dán nhãn từng nhãn một.
Nhãn nhựa thường được in trên máy in flexo cuộn khổ nhỏ, hoặc in ống đồng. Cuộn sau khi in, sẽ được chia thành cuộn nhỏ, giao cho khách hàng. Cuộn sẽ được cắt thành từng nhân và dán vào chai trên dây chuyền đóng gói tự động bằng các loại keo tương thích với bề mặt cần dán, hoặc xử lý căng với nhãn sử dụng màng căng, hay xử 

In [10]:
print(result)

page_content='Xác định các thông tin cần thiết: - Thông tin về sản phẩm:
+ Kích thước và thể tích của sản phẩm bên trong
+ Sản phẩm có cần không gian để dịch chuyển, hay cứng, hoặc có thể nén được
+ Khối lượng sản phẩm
+ Cấp độ của bao bì
+ Số lượng sản phẩm trong bao bì
+ Sản phẩm là chất khí, rắn, chất lỏng, bột hay kem: Nếu là chất khí phải quan tâm đến thể tích, áp suất khối lượng riêng, nhiệt độ. Nếu là chất lỏng cần quan tâm đến độ nhớt, dính, khả năng tạo hơi, khối lượng riêng.
+ Tính chất dễ vỡ của sản phẩm và sự chống va đập cần thiết khi vận chuyển
+ Bị hỏng khi mài mòn, va đập
+ Có bị hư hỏng khi ẩm, có cần chống ẩm hay không
+ Có mùi hay không
+ Khả năng tạo khí
+ Các yêu cầu đặc biệt để bảo quản
+ Có dầu, mỡ...
+ Nhạy cảm với nhiệt độ quá cao hoặc quá thấp
+ Thay đổi màu sắc hay tính chất khi bị chiếu sáng
+ Tính chất hóa học của sản phẩm: tính axít, tính kiềm, hoạt tính hóa học...
- Thông tin về khách hàng
+ Ai là người sử dụng (phụ nữ, đàn ông, độ tuổi, nhóm người nào)
+